#### Example: WebSearch as a Tool
1) Prompt -> LLM.
2) LLM decides a web search is needed.
3) Call WebSearch tool
4) Augment user's prompt with search results
5) Generate answer

In [7]:
from dotenv import load_dotenv
import os
from tavily import TavilyClient
from typing import List
import dspy

load_dotenv()

True

In [8]:
tavily_client = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))

def fetch_recent_news(query: str) -> List[str]:
    """
    Inputs a query string, searches for news, and returns the top results.
    """
    
    response = tavily_client.search(query=query, topic='news', max_results=4)
    return [
        x['content']
        for x in response['results']
    ]

class HaikuGenerator(dspy.Signature):
    """Generates a haiku about the latest news on the query."""
    query = dspy.InputField()
    summary = dspy.OutputField(desc='A summary of the latest news.')
    haiku = dspy.OutputField()

In [ ]:
program = dspy.ReAct(
    signature=HaikuGenerator,
    tools=[fetch_recent_news],
    max_iters=1,  # number of tool calls that the LLM can make
)
program.set_lm(dspy.LM('gemini/gemini-2.5-flash-lite', temperature=0.7))
pred = program(query='OpenAI')

In [13]:
print(pred.summary)
print()
print(pred.haiku)

Recent news about OpenAI includes Elon Musk's lawsuit alleging a violation of its founding mission, which will proceed to trial. The company is also reportedly developing a new voice AI model and devices, and has launched ChatGPT Health, a feature for analyzing medical test results and offering health advice.

Musk's suit goes to trial,
Voice AI, health insights bloom,
Future takes new form.


In [14]:
print(program.inspect_history(n=4))





[2026-01-08T13:57:25.587793]

System message:

Your input fields are:
1. `query` (str): 
2. `trajectory` (str):
Your output fields are:
1. `next_thought` (str): 
2. `next_tool_name` (Literal['fetch_recent_news', 'finish']): 
3. `next_tool_args` (dict[str, Any]):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## query ## ]]
{query}

[[ ## trajectory ## ]]
{trajectory}

[[ ## next_thought ## ]]
{next_thought}

[[ ## next_tool_name ## ]]
{next_tool_name}        # note: the value you produce must exactly match (no extra characters) one of: fetch_recent_news; finish

[[ ## next_tool_args ## ]]
{next_tool_args}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Generates a haiku about the latest news on the query.
        
        You are an Agent. In each episode, you will be given the f

Key fragment:
```ini
[[ ## next_tool_name ## ]]
fetch_recent_news

[[ ## next_tool_args ## ]]
{"query": "OpenAI"}

```

MCP Servers
- Not covered here.
- Standardized protocol of providing context and tools to LLMs.

Popularity of Tools
- Tools in Gemini CLI: `ReadFolder`, `ReadFile`, `SearchText`, `FindFiles`, `Edit`, `WriteFile`, `WebFetch`, `ReadManyFiles`, `Shell`, `SaveMemory`, `GoogleSearch`

In [15]:
class HaikuGenerator(dspy.Signature):
    """
    Generates a haiku about the latest news on the query.
    Saves the final summary in a file.
    """
    query = dspy.InputField()
    summary = dspy.OutputField(desc='A summary of the latest news.')
    haiku = dspy.OutputField()

def write_file(text: str, filename: str):
    """Write text into a file."""
    with open(filename,'w') as f:
        f.write(text)

In [ ]:
program = dspy.ReAct(
    signature=HaikuGenerator,
    tools=[fetch_recent_news, write_file],
    max_iters=2,  # need to increase this so it can call 2 tools
)
program.set_lm(dspy.LM('gemini/gemini-2.5-flash-lite', temperature=0.7))
pred = program(query='OpenAI')

In [20]:
print(pred.summary)

Recent news about OpenAI includes a lawsuit filed by Elon Musk, alleging the company violated its founding mission by becoming a for-profit entity. A judge has allowed this case to proceed to trial. Additionally, OpenAI is reportedly developing a new voice AI model for voice-based devices, with plans to potentially ship millions of units. The company has also launched ChatGPT Health, a new feature allowing users to analyze medical test results, get dietary advice, and prepare for doctor's appointments, marking a significant expansion into the healthcare sector.


In [21]:
program.inspect_history(n=4)





[2026-01-08T14:11:05.101507]

System message:

Your input fields are:
1. `query` (str): 
2. `trajectory` (str):
Your output fields are:
1. `next_thought` (str): 
2. `next_tool_name` (Literal['fetch_recent_news', 'write_file', 'finish']): 
3. `next_tool_args` (dict[str, Any]):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## query ## ]]
{query}

[[ ## trajectory ## ]]
{trajectory}

[[ ## next_thought ## ]]
{next_thought}

[[ ## next_tool_name ## ]]
{next_tool_name}        # note: the value you produce must exactly match (no extra characters) one of: fetch_recent_news; write_file; finish

[[ ## next_tool_args ## ]]
{next_tool_args}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Generates a haiku about the latest news on the query.
        Saves the final summary in a file.
      